In [ ]:
import hist
import feather
import matplotlib.pyplot as plt
import matplotlib as mpl
import mplhep as hep
import pandas as pd
import numpy as np
import pyarrow.feather as feather
from bff_processor.plotting_utils import color_fader
import re

In [ ]:
hep.set_style(hep.style.CMS)
plt.rcParams.update({
    "text.usetex": True,
})

In [ ]:
era = '2017'
if era=='2016':
    lumistr=35.50
if era=='2017':
    lumistr=41.85
if era=='2018':
    lumistr=58.88

In [ ]:
name_dict = {
'BFFZprimeToMuMu_M_125_dbs0p04': r'125 $\delta_{bs}$ 0.04', 
'BFFZprimeToMuMu_M_150_dbs0p04': r'150 $\delta_{bs}$ 0.04',
'BFFZprimeToMuMu_M_175_dbs0p04': r'175 $\delta_{bs}$ 0.04', 
'BFFZprimeToMuMu_M_200_dbs0p04': r'200 $\delta_{bs}$ 0.04',
'BFFZprimeToMuMu_M_200_dbs0p5': r'200 $\delta_{bs}$ 0.5', 
'BFFZprimeToMuMu_M_200_dbs1p0': r'200 $\delta_{bs}$ 1.0',
'BFFZprimeToMuMu_M_350_dbs0p04': r'350 $\delta_{bs}$ 0.04', 
'BFFZprimeToMuMu_M_350_dbs0p5': r'350 $\delta_{bs}$ 0.5',
'BFFZprimeToMuMu_M_350_dbs1p0': r'350 $\delta_{bs}$ 1.0', 
'BFFZprimeToMuMu_M_500_dbs0p04': r'500 $\delta_{bs}$ 0.04',
'BFFZprimeToMuMu_M_500_dbs0p5': r'500 $\delta_{bs}$ 0.5', 
'BFFZprimeToMuMu_M_500_dbs1p0': r'500 $\delta_{bs}$ 1.0',
'ZToEE_M_120_200': 'DY', 
'ZToEE_M_200_400': 'DY', 
'ZToEE_M_400_800': 'DY',
'ZToEE_M_50_120': 'DY', 
'ZToEE_M_800_1400': 'DY', 
'ZToMuMu_M_120_200': 'DY',
'ZToMuMu_M_200_400': 'DY', 
'ZToMuMu_M_400_800': 'DY', 
'ZToMuMu_M_50_120': 'DY',
'ZToMuMu_M_800_1400': 'DY', 
'data_el': 'data', 
'data_mu': 'data', 
'mc_santitop': 'ST',
'mc_stop': 'ST', 
'mc_ttbar': 'TT', 
'mc_wz': 'DB', 
'mc_zz': 'DB',
'mc_ww': 'DB'
}
def make_labels(df):
    names = df.name.to_numpy()
    for name, label in name_dict.items():
        names[names==name] = label
    return names

In [ ]:
reg_keys = {'SR[0-9]':([0,0], r'$SR_b^{\mu\mu}$'),
    'CR[0-9]0':([1,0], r'$CR_j^{\mu\mu}$'),
    'CR[0-9]3':([0,1], r'$CR_b^{ee}$'),
    'CR[0-9]4':([1,1], r'$CR_j^{ee}$')}

In [ ]:
df = feather.read_feather('data/combined_{}.feather'.format(era))

In [ ]:
df['labels'] = make_labels(df)

In [ ]:
def make_hist(
              df, 
              axismeta, axisname, 
              weightname,
              regionname,
              typename,
              variablename,
              uniquelabels=[]
             ):

    ldf = df[(df[regionname]==1) & (df.type==typename)]
    values = ldf[variablename]
    weights = ldf[weightname]
    labels = ldf['labels']
    if uniquelabels==[]:
        uniquelabels =  np.unique(labels)
    
    dr_hist = (hist.Hist.new
               .StrCat(uniquelabels, name = "labels")
               .Reg(*axismeta, name=axisname)
               .Weight())
    
    dr_hist.fill(labels, values, weight=weights)
    return dr_hist

In [ ]:
def make_type_hists(
              df, 
              axismeta, axisname, 
              weightname,
              regionname,
              variablename,
              data = 1
             ):
    uniquelabels = ['DiBoson', 'ST', 'TT', 'DY']
    typename = 'bck'
    bck_hist = make_hist(df, axismeta, axisname, weightname,regionname,typename,
                         variablename,
                         uniquelabels=uniquelabels)
    
    typename = 'sig'
    sig_hist = make_hist(df, axismeta, axisname, weightname,regionname,typename,
                        variablename,)
    
    if data:
        typename = 'data'
        data_hist = make_hist(df, axismeta, axisname, weightname,regionname,typename,
                             variablename,)
        return bck_hist, sig_hist, data_hist
    else:
        return bck_hist, sig_hist

In [ ]:
def make_sys(df,  axismeta, axisname, regiontemplate, weightname, bck_hist, sig_hist, variablename,
            regionname):  
    sys_array = []
    sig_sys_array = []
    for jetcorr in ['jer', 'jesTotal']:
        cor = []
        sig_cor = []
        for direction in ['Up','Down']:
            correction = '{}{}'.format(jetcorr, direction)
            regionnamesys = regiontemplate.format(correction)
            bck_hist_corr_temp, sig_hist_corr_tem = make_type_hists( df,  axismeta, axisname,  
                                                                    weightname, regionnamesys, 
                                                                    variablename,
                                                                    data=0)
            cor.append(bck_hist_corr_temp[::sum,:].values()-bck_hist[::sum,:].values())
            sig_cor.append(sig_hist_corr_tem.values()-sig_hist.values())
            
        cor = sorted(cor, key=lambda x: np.sum(x))
        sig_cor = sorted(sig_cor, key=lambda x: np.sum(x))
        sys_array.append(cor)
        sig_sys_array.append(sig_cor)
        
    weightsys = ["Weight_Pu","Weight_BTag","Weight_PUID","Weight_PDF_ISRFSR_","Weight_MuonSF","Weight_ElectronSF"]
    for sys in weightsys:
        cor = []
        sig_cor = []
        for direction in ['Up','Down']:
            weightnamesys = sys+direction
            bck_hist_corr_temp, sig_hist_corr_tem = make_type_hists( df,  axismeta, axisname,  
                                                                    weightnamesys, regionname, 
                                                                    variablename,
                                                                    data=0)
            cor.append(bck_hist_corr_temp[::sum,:].values()-bck_hist[::sum,:].values())
            sig_cor.append(sig_hist_corr_tem.values()-sig_hist.values())
            
        cor = sorted(cor, key=lambda x: np.sum(x))
        sig_cor = sorted(sig_cor, key=lambda x: np.sum(x))
        sys_array.append(cor)
        sig_sys_array.append(sig_cor)

    down, up = np.sum(sys_array,axis=0)
    sig_down, sig_up = np.sum(sig_sys_array, axis=0)
    return down, up, sig_down, sig_up

In [ ]:

def make_stack_plot(reg, ax, rax, lumistr, era, axismeta, axisname, variablename, fontsize=10):
    #make plots
    weightname = 'Weight'
    regiontemplate = '{}_{{}}'.format(reg)
    regionname = regiontemplate.format('nom')
    bck_hist, sig_hist, data_hist = make_type_hists( df,  axismeta, axisname,  weightname, 
                                                    regionname, 
                                                    variablename
                                                   )

    # calc sys variations for mc
    down, up, sig_down, sig_up = make_sys(df,  axismeta, axisname, 
                                          regiontemplate, weightname,
                                          bck_hist, sig_hist,
                                         variablename,
                                         regionname)
    
    ####draw plots

    # Here is an example of setting up a color cycler to color the various fill patches
    # We get the colors from this useful utility: http://colorbrewer2.org/#type=qualitative&scheme=Paired&n=6
    
    colors=['#ccffcc','#ffccff','#ffff99','#99ffff']
    bck_hist.plot1d(
        ax=ax,
        overlay = "labels",
        histtype="fill",
        stack=True,
        alpha=.9,
        edgecolor=(0, 0, 0, 0.3),
        color=colors
    )
    
    mc_total = bck_hist[::sum,:].values()
    stat_unc = bck_hist[::sum,:].variances()**.5
    ax.stairs(
        edges=bck_hist.axes[1].edges,
        baseline=mc_total + down - stat_unc,
        values=mc_total + up + stat_unc,
        hatch="///",
        label="Stat. + Sys. unc.",
        facecolor="none",
        linewidth=0,
        color="black",
    )
    if not 'SR' in reg: 
        data_hist.plot1d(
            ax=ax, 
            overlay="labels", 
            color='black',
            linestyle='none',
            histtype="errorbar"
        )
    
    
    # plot signals
    c1='#ff2f00'
    c2='#0486ff' #gr
    nsig = sig_hist.axes[0].size
    for i, (_down, _up) in enumerate(zip(sig_down, sig_up)):
        c = color_fader(c1,c2,mix=(i+.0)/nsig)
        label = sig_hist.axes[0].value(i)
        
        baseline = sig_hist[i,:].values() + _down - sig_hist[i,:].variances()**.5
        values =  sig_hist[i,:].values() + _up + sig_hist[i,:].variances()**.5
        
        if '0.04' in label:
            sig_hist[i,:].plot1d(ax=ax,color=c,label=label, yerr=values*0)
        else:
            sig_hist[i,:].plot1d(ax=ax,color=c, yerr=values*0)
        

        ax.stairs(
            edges=bck_hist.axes[1].edges,
            baseline = baseline,
            values= values,
            facecolor=c,
            linewidth=0,
            color=c,
            alpha=.25,
            fill=True
        )
    
    ax.set_yscale('log')
    ax.set_ylim(bottom=1e-1)
    leg_title = ''
    for regstr, (_,title) in reg_keys.items():
        if len(re.findall(regstr,region))>0:
            leg_title = title 
    ax.legend(ncol=2, title=leg_title, fontsize=fontsize)
    ax.set_ylabel('Count per 5 GeV')
    
    # now draw a ratio plot
    from hist.intervals import ratio_uncertainty
    if not 'SR' in reg: 
        rax.errorbar(
                data_hist.axes[1].centers,
                data_hist[::sum,:].values()/mc_total,
                yerr = data_hist[::sum,:].variances()**.5/mc_total,
                linestyle='None',
                marker='o',
                color='k'
        )
    
    #sorts values to preserve correct order
    baseline = (down - stat_unc)/mc_total+1
    values = (up + stat_unc)/mc_total+1
    comb = np.array([baseline, values]).T
    comb.sort(axis=1)
    (baseline,values) = comb.T
    rax.stairs(
        edges=bck_hist.axes[1].edges,
        baseline=baseline,
        values=values,
        hatch="///",
        facecolor="none",
        linewidth=0,
        color="black",
    )
    
    rax.axhline(y=.5, linestyle=":", color="gray")
    rax.axhline(y=1.0, linestyle="dashed", color="gray")
    rax.axhline(y=1.5, linestyle=":", color="gray")
    rax.set_xlabel(axisname)
    rax.set_ylabel("Ratio", loc="center")
    rax.set_ylim(0, 2)
    #rax.set_xlabel("Lepton $p_{T}$ [GeV]")
    
    # add some labels
    lumi = hep.cms.label(ax=ax, lumi=lumistr,year=era);

In [ ]:
axismeta = [139,110,800]
axisname = '$m_{\ell\ell}$ [GeV]'
variablename = 'DiLepMass'

In [ ]:
#axismeta = [100,0,5]
#axisname = '$\delta_{R}$'
#variablename = 'deltaR'

In [ ]:
regions = ['SR1', 'CR10', 'CR13', 'CR14', 
 'SR2', 'CR20', 'CR23', 'CR24'] 
for region in regions:
    # define a ratio plot figure
    fig, (ax, rax) = plt.subplots(2, 1, gridspec_kw=dict(height_ratios=[3, 1], hspace=.1),
                                  sharex=True, figsize=[12,15])
    make_stack_plot(region, ax, rax, lumistr, era, 
                    axismeta, axisname, variablename, fontsize=20)
    fig.savefig('stack_plots/{}/{}_{}.png'.format(era, variablename, region))

In [ ]:
from bff_processor.utils import nratio_plot_template, hist2unc, vunc2nom
def plot_reg(nJets, verbose=False):
    fig, axes = nratio_plot_template(nPlots=[2,2],rps = 2, figsize=[20,20])
    reg_keys = {'SR{}_{}':([0,0], r'$SR_b^{\mu\mu}$'),
    'CR{}0_{}':([1,0], r'$CR_j^{\mu\mu}$'),
    'CR{}3_{}':([0,1], r'$CR_b^{ee}$'),
    'CR{}4_{}':([1,1], r'$CR_j^{ee}$')}
    for name, ((i,j), region_text) in reg_keys.items():
        name = name.format(nJets, 'nom')
        reg = name.replace('_nom', '')
        print(name)
        
        top, bottom = axes[i][j]
        label='{} {} Jet'.format(region_text,nJets)
        make_stack_plot(reg, top, bottom, lumistr, era,
                        axismeta, axisname, variablename, fontsize=18)
        
    return fig

In [ ]:
fig = plot_reg(1, verbose=False)
fig.savefig('stack_plots/{}/{}_nJets_{}.png'.format(era, variablename, 1))

In [ ]:
fig = plot_reg(2, verbose=False)
fig.savefig('stack_plots/{}/{}_nJets_{}.png'.format(era, variablename, 2));